In [1]:
import requests, os, pandas as pd, glob

In [2]:
key = "7a6c64784d6a756e393572646b6d45"

url = f"http://openapi.seoul.go.kr:8088/{key}/json/CardSubwayStatsNew/{1}/{607}/{20220229}"
response = requests.get(url)
response.content.decode("utf-8")
data = response.json()
data

KeyboardInterrupt: 

In [ ]:
key = "7a6c64784d6a756e393572646b6d45"
#url = "http://openapi.seoul.go.kr:8088/"+key+"/json/CardSubwayStatsNew/1/-1/20151101"

month_list = range(1,13)
day_list = [i for i in range(1,32)]
data_list =[]
for i in month_list:
    for j in day_list:
        try:
            #돌려 보고 잘 돌아가면 2022 for문으로 수정. 귀찮으니 손으로 ㅎ자ㅏ
            day = '2022' + (str(i) if i >= 10 else '0'+str(i)) + (str(j) if j >= 10 else '0'+str(j))
            url = f"http://openapi.seoul.go.kr:8088/{key}/json/CardSubwayStatsNew/{1}/{607}/{int(day)}"
            response = requests.get(url)
            response.content.decode("utf-8")
            data = response.json()
            row_data = data['CardSubwayStatsNew']['row']
            data_list.append(row_data)
        except KeyError:
            pass
        
        
combined_df = pd.DataFrame()

for row_data in data_list:
    temp_df = pd.DataFrame(row_data)
    combined_df = pd.concat([combined_df, temp_df], ignore_index=True)

NameError: name 'requests' is not defined

In [3]:
combined_df

,USE_DT,LINE_NUM,SUB_STA_NM,RIDE_PASGR_NUM,ALIGHT_PASGR_NUM,WORK_DT
0,20220101,1호선,서울역,18398.0,16926.0,20220104
1,20220101,1호선,동묘앞,7240.0,7366.0,20220104
2,20220101,1호선,시청,5604.0,5006.0,20220104
3,20220101,1호선,종각,9977.0,8864.0,20220104
4,20220101,1호선,종로3가,11017.0,9630.0,20220104
...,...,...,...,...,...,...
28877,20221222,중앙선,아신,552.0,581.0,20221225
28878,20221222,중앙선,오빈,329.0,324.0,20221225
28879,20221222,중앙선,양평,2847.0,2982.0,20221225
28880,20221222,중앙선,원덕,278.0,272.0,20221225


In [4]:
combined_df['RIDE_PASGR_NUM'] = combined_df['RIDE_PASGR_NUM'].astype(int)
combined_df['ALIGHT_PASGR_NUM'] = combined_df['ALIGHT_PASGR_NUM'].astype(int)

In [5]:
combined_df['WORK_DT'].unique()

array(['20220104', '20220111', '20220118', '20220125', '20220204',
       '20220211', '20220218', '20220225', '20220304', '20220311',
       '20220318', '20220325', '20220404', '20220411', '20220418',
       '20220425', '20220504', '20220511', '20220518', '20220525',
       '20220604', '20220611', '20220618', '20220625', '20220704',
       '20220711', '20220718', '20220725', '20220804', '20220811',
       '20220818', '20220825', '20220904', '20220911', '20220918',
       '20220925', '20221004', '20221011', '20221018', '20221025',
       '20221104', '20221111', '20221118', '20221125', '20221204',
       '20221211', '20221218', '20221225'], dtype=object)

In [6]:
combined_df

,USE_DT,LINE_NUM,SUB_STA_NM,RIDE_PASGR_NUM,ALIGHT_PASGR_NUM,WORK_DT
0,20220101,1호선,서울역,18398,16926,20220104
1,20220101,1호선,동묘앞,7240,7366,20220104
2,20220101,1호선,시청,5604,5006,20220104
3,20220101,1호선,종각,9977,8864,20220104
4,20220101,1호선,종로3가,11017,9630,20220104
...,...,...,...,...,...,...
28877,20221222,중앙선,아신,552,581,20221225
28878,20221222,중앙선,오빈,329,324,20221225
28879,20221222,중앙선,양평,2847,2982,20221225
28880,20221222,중앙선,원덕,278,272,20221225


In [7]:
combined_df['YEAR_MONTH'] = combined_df['USE_DT'].str[:6]

# Group the data by 'YEAR_MONTH', 'LINE_NUM', and 'SUB_STA_NM' and calculate the average of 'RIDE_PASGR_NUM' and 'ALIGHT_PASGR_NUM'
summary_df = combined_df.groupby(['YEAR_MONTH', 'LINE_NUM', 'SUB_STA_NM'])[['RIDE_PASGR_NUM', 'ALIGHT_PASGR_NUM']].mean().round(0).reset_index()

# Reset the index to obtain a clean DataFrame
summary_df = summary_df.reset_index(drop=True)

summary_df['RIDE_PASGR_NUM'] = summary_df['RIDE_PASGR_NUM'].astype(int)
summary_df['ALIGHT_PASGR_NUM'] = summary_df['ALIGHT_PASGR_NUM'].astype(int)

# Now, summary_df contains the desired summary with monthly averages for each 'LINE_NUM' and 'SUB_STA_NM'
print(summary_df)


     YEAR_MONTH LINE_NUM SUB_STA_NM  RIDE_PASGR_NUM  ALIGHT_PASGR_NUM
0        202201      1호선        동대문            8963              8322
1        202201      1호선        동묘앞            9477              9802
2        202201      1호선        서울역           25790             25225
3        202201      1호선         시청            8630              8732
4        202201      1호선        신설동            8226              7747
...         ...      ...        ...             ...               ...
7272     202212      중앙선         원덕             310               302
7273     202212      중앙선         중랑            5819              5564
7274     202212      중앙선         지평              56                47
7275     202212      중앙선         팔당             760               802
7276     202212      중앙선         회기           29056             27844

[7277 rows x 5 columns]


In [11]:
summary_df

,YEAR_MONTH,LINE_NUM,SUB_STA_NM,RIDE_PASGR_NUM,ALIGHT_PASGR_NUM
0,202201,1호선,동대문,8963,8322
1,202201,1호선,동묘앞,9477,9802
2,202201,1호선,서울역,25790,25225
3,202201,1호선,시청,8630,8732
4,202201,1호선,신설동,8226,7747
...,...,...,...,...,...
7272,202212,중앙선,원덕,310,302
7273,202212,중앙선,중랑,5819,5564
7274,202212,중앙선,지평,56,47
7275,202212,중앙선,팔당,760,802


In [ ]:
#유동인구로 사용할거고
